#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Segmentology DV360 Parameters
DV360 funnel analysis using Census data.
 1. Wait for <b>BigQuery->UNDEFINED->UNDEFINED->Census_Pivot</b> to be created.
 1. Join the <a hre='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/4a908845-fdba-4023-9bb7-68666202bafb/page/K15YB/' target='_blank'>DV360 Segmentology</a>. Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to <b>BigQuery->UNDEFINED->(field:recipe_slug}->Census_Pivot</b>.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'auth_read': 'user',  # Credentials used for reading data.
  'recipe_project': '',  # Project ID hosting dataset.
  'auth_write': 'service',  # Authorization used for writing data.
  'recipe_name': '',  # Name of report, not needed if ID used.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'partners': [],  # DV360 partner id.
  'advertisers': [],  # Comma delimited list of DV360 advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Segmentology DV360
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'hour': [
        4
      ],
      'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
      'auth': 'user',
      'description': 'Create a dataset for bigquery tables.'
    }
  },
  {
    'bigquery': {
      'function': 'pearson_significance_test',
      'auth': 'user',
      'to': {
        'dataset': {'field': {'order': 4,'name': 'recipe_slug','default': '','kind': 'string','description': 'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_PARTNER': {
            'values': {'field': {'order': 5,'name': 'partners','default': [],'kind': 'integer_list','description': 'DV360 partner id.'}}
          },
          'FILTER_ADVERTISER': {
            'values': {'field': {'order': 6,'name': 'advertisers','default': [],'kind': 'integer_list','description': 'Comma delimited list of DV360 advertiser ids.'}}
          }
        },
        'body': {
          'schedule': {
            'frequency': 'WEEKLY'
          },
          'timezoneCode': {'field': {'default': 'America/Los_Angeles','name': 'recipe_timezone','description': 'Timezone for report dates.','kind': 'timezone'}},
          'metadata': {
            'format': 'CSV',
            'title': {'field': {'name': 'recipe_name','description': 'Name of report, not needed if ID used.','kind': 'string','prefix': 'Segmentology ','order': 3,'default': ''}},
            'dataRange': 'LAST_30_DAYS'
          },
          'params': {
            'metrics': [
              'METRIC_BILLABLE_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS'
            ],
            'type': 'TYPE_CROSS_PARTNER',
            'groupBys': [
              'FILTER_PARTNER',
              'FILTER_PARTNER_NAME',
              'FILTER_ADVERTISER',
              'FILTER_ADVERTISER_NAME',
              'FILTER_MEDIA_PLAN',
              'FILTER_MEDIA_PLAN_NAME',
              'FILTER_ZIP_POSTAL_CODE'
            ]
          }
        }
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'name': {'field': {'name': 'recipe_name','description': 'Name of report, not needed if ID used.','kind': 'string','prefix': 'Segmentology ','order': 3,'default': ''}}
      },
      'out': {
        'bigquery': {
          'dataset': {'field': {'order': 4,'name': 'recipe_slug','default': '','kind': 'string','description': 'Name of Google BigQuery dataset to create.'}},
          'auth': 'user',
          'schema': [
            {
              'mode': 'REQUIRED',
              'name': 'Partner_Id',
              'type': 'INTEGER'
            },
            {
              'mode': 'REQUIRED',
              'name': 'Partner',
              'type': 'STRING'
            },
            {
              'mode': 'REQUIRED',
              'name': 'Advertiser_Id',
              'type': 'INTEGER'
            },
            {
              'mode': 'REQUIRED',
              'name': 'Advertiser',
              'type': 'STRING'
            },
            {
              'mode': 'REQUIRED',
              'name': 'Campaign_Id',
              'type': 'INTEGER'
            },
            {
              'mode': 'REQUIRED',
              'name': 'Campaign',
              'type': 'STRING'
            },
            {
              'mode': 'NULLABLE',
              'name': 'Zip',
              'type': 'STRING'
            },
            {
              'mode': 'NULLABLE',
              'name': 'Impressions',
              'type': 'FLOAT'
            },
            {
              'mode': 'NULLABLE',
              'name': 'Clicks',
              'type': 'INTEGER'
            },
            {
              'mode': 'NULLABLE',
              'name': 'Conversions',
              'type': 'FLOAT'
            }
          ],
          'table': 'DV360_KPI'
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be written in BigQuery.','kind': 'string'}},
        'view': 'DV360_KPI_Normalized'
      },
      'from': {
        'legacy': False,
        'query': 'SELECT            Partner_Id,            Partner,            Advertiser_Id,            Advertiser,            Campaign_Id,            Campaign,            Zip,            SAFE_DIVIDE(Impressions, SUM(Impressions) OVER(PARTITION BY Advertiser_Id)) AS Impression_Percent,            SAFE_DIVIDE(Clicks, Impressions) AS Click_Percent,            SAFE_DIVIDE(Conversions, Impressions) AS Conversion_Percent,            Impressions AS Impressions          FROM            `{project}.{dataset}.DV360_KPI`;        ',
        'parameters': {
          'dataset': {'field': {'name': 'recipe_slug','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
          'project': {'field': {'name': 'recipe_project','description': 'Project ID hosting dataset.','kind': 'string'}}
        }
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'normalize': {
        'census_geography': 'zip_codes',
        'census_span': '5yr',
        'census_year': '2018'
      },
      'to': {
        'dataset': {'field': {'order': 4,'name': 'recipe_slug','default': '','kind': 'string','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'correlate': {
        'correlate': [
          'Impression_Percent',
          'Click_Percent',
          'Conversion_Percent'
        ],
        'pass': [
          'Partner_Id',
          'Partner',
          'Advertiser_Id',
          'Advertiser',
          'Campaign_Id',
          'Campaign'
        ],
        'sum': [
          'Impressions'
        ],
        'dataset': {'field': {'order': 4,'name': 'recipe_slug','default': '','kind': 'string','description': 'Name of Google BigQuery dataset to create.'}},
        'significance': 80,
        'join': 'Zip',
        'table': 'DV360_KPI_Normalized'
      },
      'to': {
        'dataset': {'field': {'order': 4,'name': 'recipe_slug','default': '','kind': 'string','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
